<img src="logo.png">

# Los mamíferos y sus hábitos de sueño

La tabla ``msleep.csv``, que puedes descargar aquí, contiene información sobre 83 mamíferos diferentes. La información tiene su nombre común, su género, su tipo de dieta, su orden taxológico, su estado de conservación, así como las horas de sueño al día que duermen, las horas que corresponden al sueño profundo (REM), cuánto dura su [ciclo de sueño](https://www.nhlbi.nih.gov/es/salud/sueno/estadios-del-sueno#:~:text=Al%20dormir%2C%20se%20atraviesa%20un,entre%20un%20ciclo%20y%20otro.), cuántas horas se mantiene despierto al día, su masa cerebral (en Kg) y su masa corporal (también en kg).

Resuelve los siguientes problemas:

In [ ]:
# Cargado de bibliotecas y lectura del archivo

import pandas as pd
import numpy as np
from siuba import *
from siuba.dply.vector import *
from plotnine import *

sp = pd.read_csv('msleep.csv')

In [ ]:
# Cambia el nombre de cada columna a su traducción en español.

sp = (sp >> rename(nombre = "name",
                   género = "genus",
                   dieta = "vore",
                   orden = "order",
                   conservacion = "conservation",
                   horas_sueño = "sleep_total",
                   horas_rem = "sleep_rem",
                   ciclo_sueño = "sleep_cycle",
                   horas_despierto = "awake",
                   masa_cerebral = "brainwt",
                   masa_corporal = "bodywt")
     )

In [ ]:
# Ordena la tabla alaféticamente por dieta, orden y nombre, y muestra el género de los tres primeros renglones. 

sp >> arrange(_.dieta,_.orden,_.nombre) >> head(3) >> select(_.género)

In [ ]:
# ¿Cuál es la dieta más común en la tabla? ¿Cuántos omnívoros hay?

sp >> group_by(_.dieta) >> summarize(total = n(_)) >> arrange(-_.total)

In [ ]:
# ¿Cuál es el orden más común en la tabla?  ¿Hay más carnívora que primates?

sp >> group_by(_.orden) >> summarize(total = n(_)) >> arrange(-_.total)

In [ ]:
# ¿Cuáles son los mamíferos que más y menos duermen?

(sp >> filter(_.orden == "Primates") >> 
     filter((_.horas_sueño==_.horas_sueño.max()) | (_.horas_sueño==_.horas_sueño.min()))
)

In [ ]:
# ¿Qué resultado esperarías obtener si sumas las horas de sueño con las horas despierto en cada renglón?
# Calcula una nueva columna llamada "total_despierto_dormido" que sea esta suma.

sp >> mutate(total_sueño_despierto = _.horas_sueño + _.horas_despierto)

In [ ]:
# Añade el porcentaje de masa que representa el cerebro respecto del cuerpo de cada animal y responde:

sp_ratio = sp >> mutate(per_br_bd = 100*(_.masa_cerebral)/_.masa_corporal)
sp_ratio

In [ ]:
#### ¿Qué animales tienen el mayor y menor porcentaje de masa cerebral respecto a la corporal?

sp_ratio >> filter((_.per_br_bd == _.per_br_bd.max()) | (_.per_br_bd == _.per_br_bd.min())) >> arrange(-_.per_br_bd)

In [ ]:
#### ¿Cuántos animales tienen dicho porcentaje mayor al del humano?

per_br_bd_human = sp_ratio[sp_ratio["nombre"] == "Human"]["per_br_bd"].iloc[0]
per_br_bd_human

In [ ]:
sp_ratio >> filter(_.per_br_bd > per_br_bd_human) >> count()

In [ ]:
# Añade una columna que indique si el animal duerme 8 o menos horas.

sp_ratio = sp_ratio >> mutate(ocho_horas = np.where(_.horas_sueño <= 8,"Menos de 8 horas de sueño","Más de 8 horas de sueño"))
sp_ratio

In [ ]:
sp_ratio_conocido = (sp_ratio >> 
           filter(_.per_br_bd.isna() == False)           
          )

In [ ]:
# Crea un gráfico de barras donde se indique cuántos animales de cada dieta tienen mayor relación cerebro/cuerpo respecto 
# de los humanos, separando aquellos que duermen menos de 8 horas de los que duermen mas de 8 horas.

sp_ratio_conocido = sp_ratio_conocido >> mutate(mayor_humano = np.where(_.per_br_bd > per_br_bd_human,"Sí","No"))
sp_ratio_conocido

In [ ]:
resumen = sp_ratio_conocido >> group_by(_.dieta,_.mayor_humano,_.ocho_horas) >> count()
resumen

In [ ]:
(ggplot() +
    geom_col(data = resumen,
             mapping = aes(x="dieta",y="n",fill="mayor_humano"),
             position = position_dodge(preserve = "single")) +
    geom_text(data = resumen,
             mapping = aes(x="dieta",y="n+0.5",fill="mayor_humano",label="n"),
             position = position_dodge(width = 0.9)
             ) +
 facet_grid("ocho_horas~.")
)